In [83]:
import pandas as pd
import pdb
from bs4 import BeautifulSoup
import logging

In [84]:
#set-up log
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='processor.log', mode='a')
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [ ]:
#set-up output file for card supplies

#set-up output file for features - i.e. number of cards at the end of each round


In [85]:
dir_name = 'DominionPlayerLogs/2012-all/20121231'
#html_doc = 'game-20121231-000041-13409c79.html'
html_doc = 'game-20121231-002653-e5872fde.html'
#html_doc = 'game-20121231-000155-2795fdc1.html'

logging.debug("Processing file: " + str(dir_name + '/' + html_doc))

DEBUG:root:Processing file: DominionPlayerLogs/2012-all/20121231/game-20121231-002653-e5872fde.html


In [86]:
file = open(dir_name + '/' + html_doc, mode = 'r')
soup = BeautifulSoup(file, 'html.parser')

print(soup.prettify())

<html>
 <head>
  <link href="/semistatic/log.css" rel="stylesheet"/>
  <title>
   Dominion Game #259154
  </title>
 </head>
 <body>
  <pre>Intense wins!
<span class="card-curse">Curses</span>, <span class="card-none">Crossroads</span>, and <span class="card-treasure">Ill-Gotten Gains</span> are all gone.

cards in supply: <span cardname="Colony" class="card-victory">Colony</span>, <span cardname="Conspirator" class="card-none">Conspirator</span>, <span cardname="Council Room" class="card-none">Council Room</span>, <span cardname="Crossroads" class="card-none">Crossroads</span>, <span cardname="Embargo" class="card-none">Embargo</span>, <span cardname="Forge" class="card-none">Forge</span>, <span cardname="Ill-Gotten Gains" class="card-treasure">Ill-Gotten Gains</span>, <span cardname="Militia" class="card-none">Militia</span>, <span cardname="Platinum" class="card-treasure">Platinum</span>, <span cardname="Scout" class="card-none">Scout</span>, <span cardname="Spy" class="card-none">Sp

In [87]:
#convert html contents into a list of tabs, navigable strings etc
pre_tag = soup.find('pre')
pre_tag.contents
print(pre_tag.contents)

['Intense wins!\n', <span class="card-curse">Curses</span>, ', ', <span class="card-none">Crossroads</span>, ', and ', <span class="card-treasure">Ill-Gotten Gains</span>, ' are all gone.\n\ncards in supply: ', <span cardname="Colony" class="card-victory">Colony</span>, ', ', <span cardname="Conspirator" class="card-none">Conspirator</span>, ', ', <span cardname="Council Room" class="card-none">Council Room</span>, ', ', <span cardname="Crossroads" class="card-none">Crossroads</span>, ', ', <span cardname="Embargo" class="card-none">Embargo</span>, ', ', <span cardname="Forge" class="card-none">Forge</span>, ', ', <span cardname="Ill-Gotten Gains" class="card-treasure">Ill-Gotten Gains</span>, ', ', <span cardname="Militia" class="card-none">Militia</span>, ', ', <span cardname="Platinum" class="card-treasure">Platinum</span>, ', ', <span cardname="Scout" class="card-none">Scout</span>, ', ', <span cardname="Spy" class="card-none">Spy</span>, ', and ', <span cardname="Woodcutter" class

In [144]:
#first extract number of players and final score
#look for entries of the form #n <name>: x points' - these are tagged with a 'b'
players = []
scores = []
list_b_tags = soup.find_all('b');
for tag in list_b_tags:
    s = tag.text
    if '#' in s:
        #extract name (assume string of form '#n ' comes before the name)
        players.append(s[3:])
        #next sibing contains the score, assume score is after a string of the form ': ' and is 2 digits long
        score_text = tag.next_sibling
        scores.append(score_text[2:4])
print(players)
print(scores)


['Intense', 'faulkner']
['7 ', '2 ']


In [160]:
#next extract kingdom cards - first we deal with the case where the default set is chosen and players dont 
#make changes

#iterate through contents list under we find an element which contains 'cards in supply'. 
default_supply_cards_start_index = 0
for i, s in enumerate(pre_tag.contents):
    if 'cards in supply' in s:
        default_supply_cards_start_index = i
        break

#there is an initial default set of supply cards. This may be modified later by player choice if this default 
#selection is not chosen. So first determine if the players use the default set
default_supply_set = False
for s in pre_tag.contents[default_supply_cards_start_index:]:
    if 'Default card selection was used' in s:
        default_supply_set = True
        break
    elif '----------------------' in s:
        break
       
#if the default set was used then grab card names
supply_cards = []
if default_supply_set == True:
    for s in pre_tag.contents[default_supply_cards_start_index:]:
        if 'Default card selection was used' in s:
            break
        else:
            if s.name != None:
                supply_cards.append(s.attrs['cardname'])

print("Default supply cards chosen: " + str(default_supply_set))                
print(len(supply_cards))
print(supply_cards)

Default supply cards chosen: False
0
[]


In [161]:
#next we extract supply cards when players make a choice. In this case we look for the substring 'chosen cards are'
#which tags the start of the set of chosen supply cards. This should be followed by a number of veto statements
#dependent on the number of players
index_end = 0 
if default_supply_set == False:
    for (i,s) in enumerate(pre_tag.contents[default_supply_cards_start_index:]):
        if 'chosen cards are' in s:
            #loop over chosen supply cards
            for (j,s_) in enumerate(pre_tag.contents[default_supply_cards_start_index + i + 1:]):
                if s_.name != None:
                    #pdb.set_trace()
                    supply_cards.append(s_.attrs['cardname'])
                elif '.' in s_:
                    #list of supply cards ends with a full stop
                    index_end = default_supply_cards_start_index + i + j + 1
                    break

#next each player can veto a card
vetoed_cards = []
for player in players:
    for s in pre_tag.contents[index_end:]:
        check_string = player + ' vetoes'
        if check_string in s:
            vetoed_cards.append(s.next_sibling.text)

#remove vetoed cards from list of supply cards
supply_cards = list(filter(lambda x: x not in vetoed_cards, supply_cards))

print(vetoed_cards)             
print(supply_cards)

['Scrying Pool', 'Black Market']
['Conspirator', 'Council Room', 'Crossroads', 'Embargo', 'Forge', 'Ill-Gotten Gains', 'Militia', 'Scout', 'Spy', 'Woodcutter']


In [12]:
#print(soup.title)
#print(soup.head)
#print(soup.children)
#children = soup.children
#print(next(children))
#print(soup.get_text())
#soup.find_all('span')